In [1]:
#importing libraries
import pandas as pd
import os
import numpy as np
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from keras.layers import Dense, Conv1D, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import StratifiedKFold
from keras.backend import dropout
import keras
import random
from sklearn.model_selection import train_test_split
from tensorflow.keras import regularizers

2024-06-30 20:39:19.570926: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-30 20:39:20.182684: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-06-30 20:39:20.182735: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2024-06-30 20:39:20.182740: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
# Set a seed value
seed_value= 1

# 1. Set `PYTHONHASHSEED` environment variable at a fixed value
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set `python` built-in pseudo-random generator at a fixed value
random.seed(seed_value)

# 3. Set `numpy` pseudo-random generator at a fixed value
np.random.seed(seed_value)

# 4. Set `tensorflow` pseudo-random generator at a fixed value
tf.random.set_seed(seed_value)

tf.keras.utils.set_random_seed(seed_value)

# 5. For layers that introduce randomness like dropout, make sure to set seed values 
# model.add(Dropout(0.25, seed=seed_value))

In [3]:
#import relevant datasets
X_train = pd.read_csv("finalXtrain.csv")
y_train = pd.read_csv("finalYtrain.csv")
X_test = pd.read_csv("finalXtest.csv")
y_test = pd.read_csv("finalYtest.csv")
X_val = pd.read_csv("finalXval.csv")
y_val = pd.read_csv("finalYval.csv")

In [4]:
#check shape
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
print(X_val.shape)
print(y_val.shape)

(191, 72)
(191, 2)
(41, 72)
(41, 2)
(42, 72)
(42, 2)


In [5]:
#build model

def define_model(seedval):
    inputs = tf.keras.Input(shape=(72, 1), name='input')
    x = Conv1D(filters=5, kernel_size=3,strides =3)(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation(tf.keras.activations.relu)(x)
    x = tf.keras.layers.MaxPooling1D(2)(x)
    x = Conv1D(filters=45, kernel_size=5,strides =2)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation(tf.keras.activations.relu)(x)
    x = tf.keras.layers.MaxPooling1D(2)(x)
    x = Flatten()(x)
    x = Dropout(0.5,seed=seedval)(x)
    x = Dense(20,activation ='relu')(x)
#     x = Dropout(0.5,seed=seedval)(x)
#     x = Dense(10,activation ='relu')(x)
#     x = Dense(5,activation ='relu')(x)
    outputs = Dense(2, activation='softmax', name='predictions')(x)
    cnn_model = tf.keras.Model(inputs=inputs, outputs=outputs)
    cnn_model.compile(loss='categorical_crossentropy', optimizer= Adam(), metrics=['accuracy'])
    return cnn_model

In [6]:
#earlystopping
es = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=50)
checkpoint_path = 'alaarg_oct9_check.h5'
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                   monitor='val_accuracy',
                                   save_best_only=True,
                                   mode='max',
                                   verbose=1)

In [7]:
model = define_model(seed_value)
print(model.summary())

2024-06-30 20:39:20.693360: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 72, 1)]           0         
                                                                 
 conv1d (Conv1D)             (None, 24, 5)             20        
                                                                 
 batch_normalization (BatchN  (None, 24, 5)            20        
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 24, 5)             0         
                                                                 
 max_pooling1d (MaxPooling1D  (None, 12, 5)            0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 4, 45)             1170  

2024-06-30 20:39:20.693663: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-06-30 20:39:20.702444: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-06-30 20:39:20.702653: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-06-30 20:39:20.702818: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-06-30 20:39:20.702976: I tensorflow/compiler/xla/stream_executo

In [8]:

history = model.fit(X_train, y_train,
              batch_size=10,
              verbose=1,
              epochs=500,
              validation_data = (X_val,y_val), 
              callbacks = [es,model_checkpoint]
                   )

Epoch 1/500


2024-06-30 20:39:23.470005: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8500
2024-06-30 20:39:23.643447: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-06-30 20:39:23.708791: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-06-30 20:39:23.710268: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7f188b72c630 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-06-30 20:39:23.710281: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA RTX A6000, Compute Capability 8.6
2024-06-30 20:39:23.710285: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (1): NVIDIA RTX A6000, Compute Capability 8.6
2024-06-30 20:39:23.714247: I tensorflow/compiler/mlir/tensorflow/uti

16/20 [=======================>......] - ETA: 0s - loss: 1.1110 - accuracy: 0.4688 
Epoch 1: val_accuracy improved from -inf to 0.30952, saving model to alaarg_oct9_check.h5
20/20 [==============================] - 3s 15ms/step - loss: 1.1548 - accuracy: 0.4712 - val_loss: 0.7258 - val_accuracy: 0.3095
Epoch 2/500
16/20 [=======================>......] - ETA: 0s - loss: 1.0074 - accuracy: 0.5688
Epoch 2: val_accuracy improved from 0.30952 to 0.54762, saving model to alaarg_oct9_check.h5
20/20 [==============================] - 0s 6ms/step - loss: 0.9421 - accuracy: 0.5864 - val_loss: 0.6956 - val_accuracy: 0.5476
Epoch 3/500
18/20 [==========================>...] - ETA: 0s - loss: 0.8050 - accuracy: 0.5833
Epoch 3: val_accuracy improved from 0.54762 to 0.71429, saving model to alaarg_oct9_check.h5
20/20 [==============================] - 0s 6ms/step - loss: 0.7976 - accuracy: 0.5759 - val_loss: 0.6719 - val_accuracy: 0.7143
Epoch 4/500
16/20 [=======================>......] - ETA: 0s -

Epoch 30/500
16/20 [=======================>......] - ETA: 0s - loss: 0.4310 - accuracy: 0.7875
Epoch 30: val_accuracy did not improve from 0.83333
20/20 [==============================] - 0s 5ms/step - loss: 0.4210 - accuracy: 0.8010 - val_loss: 0.5800 - val_accuracy: 0.6905
Epoch 31/500
16/20 [=======================>......] - ETA: 0s - loss: 0.3868 - accuracy: 0.8188
Epoch 31: val_accuracy did not improve from 0.83333
20/20 [==============================] - 0s 5ms/step - loss: 0.3822 - accuracy: 0.8272 - val_loss: 0.5905 - val_accuracy: 0.7143
Epoch 32/500
17/20 [========================>.....] - ETA: 0s - loss: 0.3763 - accuracy: 0.8176
Epoch 32: val_accuracy did not improve from 0.83333
20/20 [==============================] - 0s 5ms/step - loss: 0.3756 - accuracy: 0.8220 - val_loss: 0.5999 - val_accuracy: 0.6905
Epoch 33/500
16/20 [=======================>......] - ETA: 0s - loss: 0.3849 - accuracy: 0.8062
Epoch 33: val_accuracy did not improve from 0.83333
20/20 [==============

Epoch 60/500
16/20 [=======================>......] - ETA: 0s - loss: 0.3152 - accuracy: 0.8438
Epoch 60: val_accuracy did not improve from 0.83333
20/20 [==============================] - 0s 5ms/step - loss: 0.3096 - accuracy: 0.8534 - val_loss: 0.6819 - val_accuracy: 0.6905
Epoch 61/500
16/20 [=======================>......] - ETA: 0s - loss: 0.2527 - accuracy: 0.9187
Epoch 61: val_accuracy did not improve from 0.83333
20/20 [==============================] - 0s 5ms/step - loss: 0.2482 - accuracy: 0.9215 - val_loss: 0.7090 - val_accuracy: 0.7143
Epoch 62/500
15/20 [=====================>........] - ETA: 0s - loss: 0.2710 - accuracy: 0.8600
Epoch 62: val_accuracy did not improve from 0.83333
20/20 [==============================] - 0s 5ms/step - loss: 0.2915 - accuracy: 0.8586 - val_loss: 0.7052 - val_accuracy: 0.6905
Epoch 63/500
15/20 [=====================>........] - ETA: 0s - loss: 0.3100 - accuracy: 0.8733
Epoch 63: val_accuracy did not improve from 0.83333
20/20 [==============

In [9]:
#test model
testmodel = tf.keras.models.load_model("alaarg_oct9_check.h5")
testmodel.evaluate(X_test,y_test)

2/2 [==============================] - 0s 9ms/step - loss: 0.5750 - accuracy: 0.7805


[0.5750277042388916, 0.7804877758026123]

In [10]:
#test model
testmodel1 = tf.keras.models.load_model("alaarg_oct9_check.h5")
testmodel1.evaluate(X_val,y_val)

2/2 [==============================] - 0s 3ms/step - loss: 0.5278 - accuracy: 0.8333


[0.5277988910675049, 0.8333333134651184]

In [11]:
#test model
testmodel1 = tf.keras.models.load_model("alaarg_oct9_check.h5")
testmodel1.evaluate(X_train,y_train)

6/6 [==============================] - 0s 3ms/step - loss: 0.3992 - accuracy: 0.8639


[0.39916911721229553, 0.8638743162155151]

In [14]:
y=testmodel.predict(X_test)
#class-wise accuracy
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(y_test.to_numpy().argmax(axis=1), y.argmax(axis=1))
matrix.diagonal()/matrix.sum(axis=1)

2/2 [==============================] - 0s 2ms/step


array([0.85      , 0.76190476])